Initialize workflow.

In [ ]:
import datetime
import struct, socket
import shutil
import numpy as np
import pandas as pd
import linecache, bisect
import csv, json
import operator
import os, time, subprocess 
from collections import OrderedDict

try:
    import ipywidgets as widgets # For jupyter/ipython >= 1.4
except ImportError:
    from IPython.html import widgets

from IPython.display import display, Javascript, clear_output

path = os.getcwd().split("/") 
date = path[len(path)-1]   
dsource = path[len(path)-2]  
cpath = '/'.join(['context' if var == 'ipynb' else var for var in path][:len(path)-2]) + '/'

coff = 250;
nwloc = cpath + 'networkcontext.csv' 
srcdict,srclist = {},[]
dstdict,dstlist = {},[]
sportdict,sportlist = {},[]
dportdict,dportlist = {},[]
global svals
svals = [] 

In [ ]:
def apply_css_to_select(select):
    select._css = (
        (None, 'height', '90%'),
        (None, 'width', '90%'),
        ('select', 'overflow-x', 'auto'),
        ('select', 'width', '100%'),
        ('select', 'margin', 0)
    )

# Source IP box
scrIpLalbel = widgets.HTML(value="Source IP:", height='10%', width='100%')
# srcselect = widgets.Select(options=srclist, height='90%')
srcselect = widgets.Select(height='100%')
apply_css_to_select(srcselect)
srcIpBox = widgets.Box(width='25%', height='100%')
srcIpBox.children = (scrIpLalbel, srcselect)

# Destination IP box
dstIpLalbel = widgets.HTML(value="Dest IP:", height='10%', width='100%')
dstselect = widgets.Select(height='100%')
# dstselect = widgets.Select(options=dstlist, height='90%')
apply_css_to_select(dstselect)
dstIpBox = widgets.Box(width='25%', height='100%')
dstIpBox.children = (dstIpLalbel, dstselect)

# Source Port box
scrPortLalbel = widgets.HTML(value="Src Port:", height='10%', width='100%')
sportselect = widgets.Select(height='100%')
# sportselect = widgets.Select(options=sportlist, height='90%')
apply_css_to_select(sportselect)
srcPortBox = widgets.Box(width='20%', height='100%')
srcPortBox.children = (scrPortLalbel, sportselect)

# Destionation Port box
dstPortLalbel = widgets.HTML(value="Dst Port:", height='10%', width='100%')
dportselect = widgets.Select(height='100%')
# dportselect = widgets.Select(options=dportlist,height='90%')
apply_css_to_select(dportselect)
dstPortBox = widgets.Box(width='20%', height='100%')
dstPortBox.children = (dstPortLalbel, dportselect)

# Quick Search and Actions Box
emptyLalbel = widgets.HTML(value="&nbsp;")
srctext = widgets.Text(value='', width='100%', placeholder='Quick IP scoring')
srctext._css = (
    (None, 'width', '100%'),
)
ratingbut = widgets.RadioButtons(description='Rating:',options=['1', '2', '3'], width='100%')
assignbut = widgets.Button(description='Score', width='45%')
assignbut.button_style = 'primary'
updatebut = widgets.Button(description='Save', width='45%')
updatebut.button_style = 'primary'
updatebut._css = (
    (None, 'margin-left', '10%'),
)
actionsBox = widgets.Box(width='20%', height='100%')
actionsBox.children = (emptyLalbel, srctext,ratingbut,assignbut,updatebut)

# Container Box
bigBox = widgets.HBox(width='90%', height=250)
bigBox.children = (srcIpBox, dstIpBox, srcPortBox, dstPortBox, actionsBox)


def fill_list(list_control,source):
    options_list = ['- Select -'] 
    options_list.extend([s for s in source])
    list_control.options = options_list


def data_loader():
    # build dict of IP addresses
    #sev,score, tstart,srcIP,dstIP,sport,dport,proto,ipkt,ibyt
    srcdict,srclist = {},[]
    dstdict,dstlist = {},[]
    sportdict,sportlist = {},[]
    dportdict,dportlist = {},[] 
    global svals 
    srcips,srcports = [],[]
    dstips,dstports = [],[]
    response = GraphQLClient.request(
        query="""query($date:SpotDateType!) {
                flow{
                    suspicious(date:$date){
                    srcIP: srcIp
                    sport: srcPort
                    dstIP: dstIp
                    dport: dstPort
                }
            }
        }""",
        variables={
            'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
        }
    )
    
    
    for item in svals:
        srcips.append(item[0])
        srcports.append(item[2]) 
        dstips.append(item[1]) 
        dstports.append(item[3])
        
    rowct = 1
     
    
    if not 'errors' in response:
        for row in response['data']['flow']['suspicious']:
            if row['srcIP'] not in srcdict and row['srcIP'] not in srcips:
                srclist.append(row['srcIP'])
                srcdict[row['srcIP']] = struct.unpack("!L", socket.inet_aton(row['srcIP']))[0]
            if row['dstIP'] not in dstdict and row['dstIP'] not in dstips:
                dstlist.append(row['dstIP'])
                dstdict[row['dstIP']] = struct.unpack("!L", socket.inet_aton(row['dstIP']))[0]
            if row['sport'] not in sportdict and str(row['sport']) not in srcports:
                sportlist.append(str(row['sport']))
                sportdict[row['sport']] = row['sport']
            if row['dport'] not in dportdict and str(row['dport']) not in dstports:
                dportlist.append(str(row['dport']))
                dportdict[row['dport']] = row['dport']                
            if rowct == coff:
                break;
            rowct += 1
            
    fill_list(srcselect,srclist)
    fill_list(dstselect,dstlist)
    fill_list(sportselect,sportlist)
    fill_list(dportselect,dportlist)
            
    srcselect.value = "- Select -"
    dstselect.value = "- Select -"    
    sportselect.value = "- Select -"    
    dportselect.value = "- Select -"

In [ ]:
display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
data_loader() 
display(bigBox)

In [ ]:
def assign_score(b):
    clear_output()
    #Gets input values
    global svals
    if srctext.value != '':
        svals.append([srctext.value,dstselect.value,sportselect.value,dportselect.value, ratingbut.value])
        svals.append([srcselect.value,srctext.value,sportselect.value,dportselect.value, ratingbut.value])
        display(Javascript("$(\"option[data-value='" + srctext.value +"']\").remove();"))
        dstselect.value = "- Select -"        
        srcselect.value = "- Select -"        
    else:
        svals.append([srcselect.value,dstselect.value,sportselect.value,dportselect.value, ratingbut.value])
    
    if srcselect.value != "- Select -":  
        display(Javascript("$(\"select.widget-listbox:eq(0) option[data-value='" + srcselect.value +"']\").remove();"))  
        srcselect.value = "- Select -"
    if dstselect.value != "- Select -":
        display(Javascript("$(\"select.widget-listbox:eq(1) option[data-value='" + dstselect.value +"']\").remove();"))  
        dstselect.value = "- Select -"
    if sportselect.value != "- Select -": 
        display(Javascript("$(\"select.widget-listbox:eq(2) option[data-value='" + sportselect.value +"']\").remove();"))  
        sportselect.value = "- Select -"
    if dportselect.value != "- Select -": 
        display(Javascript("$(\"select.widget-listbox:eq(3) option[data-value='" + dportselect.value +"']\").remove();"))  
        dportselect.value = "- Select -"
    srctext.value = ""
    
    data_loader()
    
    print "Click the 'Save' button when you're finished scoring" 


def savesort(b):
    global svals
    clear_output()    

    variables = []
    mutation="""mutation($input:[NetflowScoreInputType!]!)
            {
              flow{
                score(input:$input)
                    {success}
              }
            }"""

    for row in svals:
        variables.append({
            'date': datetime.datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d'),
            'score': row[4],
            'srcIp': row[0] if row[0] != '- Select -' else None,
            'dstIp': row[1] if row[1] != '- Select -' else None,
            'srcPort': row[2] if row[2] != '- Select -' else None,
            'dstPort': row[3]  if row[3] != '- Select -' else None
            })

    var = {'input':variables}
    response = GraphQLClient.request(mutation,var)

    svals = []
    if not 'errors' in response :
        display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
        data_loader() 
        display(bigBox)
        display(Javascript('reloadParentData();'))         
        print "Suspicious connects successfully updated"        
    else:
        print "An error ocurred: " + response['errors'][0]['message']

assignbut.on_click(assign_score)
updatebut.on_click(savesort)